In [69]:
import pandas as pd 
from utils import read_df, save_df_parquet, sample_series
import ast
import nltk

In [70]:
df: pd.DataFrame = read_df('dados_tokenizados.csv')

In [71]:
df.sample(3)

,texto_original,categoria,texto_limpo,tokens,tokens_clean
3812,Outra colocação seria também a creche para ido...,Assistência Social,outra colocação seria também a creche para ido...,"['outra', 'colocação', 'seria', 'também', 'a',...","['outra', 'colocacao', 'creche', 'idosos', 're..."
2244,Revisar - Triplicar o número de beneficiários ...,Direitos Humanos e Cidadania,revisar - triplicar o número de beneficiários ...,"['revisar', '-', 'triplicar', 'o', 'número', '...","['revisar', 'triplicar', 'numero', 'beneficiar..."
461,Investir na melhoria de limpeza dos Córregos.,Zeladoria Urbana e melhorias de bairro,investir na melhoria de limpeza dos córregos.,"['investir', 'na', 'melhoria', 'de', 'limpeza'...","['investir', 'melhoria', 'limpeza', 'corregos']"


Para construir a bag of words vou precisar apenas dos tokens e da classificacao.

Terei que fazer o explode na classificação - pois um mesmo texto pode ter mais de um tema

In [72]:
df = df[['tokens_clean', 'categoria']].copy()

In [73]:
df.dtypes

tokens_clean    object
categoria       object
dtype: object

In [74]:
type(df['tokens_clean'][0])

str

In [75]:
df.head()

,tokens_clean,categoria
0,"['gente', 'sente', 'falta', 'metas', 'especifi...",Meio Ambiente
1,"['segunda', 'preocupacao', 'questao', 'area', ...",Meio Ambiente
2,"['acho', 'primeira', 'manifestacao', 'coletivo...",Outros temas
3,"['finaliza', 'gostaria', 'dizer', 'esperamos',...",Outros temas
4,"['tempos', 'pandemias', 'disse', 'primeiro', '...",Outros temas


In [76]:
#o csv abre a lista como se fosse uma string
def parse_lst_column(lst_str)->list[str]:

    if type(lst_str) is not str:
        #bizarro - mas alguns registros estao como listas ou como arrays mesmo
        lst_str = f"[{', '.join(lst_str)}]"

    lst_str = lst_str.lstrip('array')

    return ast.literal_eval(lst_str)

In [77]:
df['tokens_clean'] = df['tokens_clean'].apply(parse_lst_column)

In [78]:
type(df['tokens_clean'][0])

list

In [79]:
from sklearn.feature_extraction.text import CountVectorizer

In [80]:
df['corpora'] = df['tokens_clean'].apply(lambda x: ' '.join(x))

In [81]:
df.sample(3)

,tokens_clean,categoria,corpora
1320,"[os, sabemos, centro, acolhida, aqui, regiao, ...",Assistência Social,os sabemos centro acolhida aqui regiao centro ...
284,"[ultimo, vou, vou, colocar, escrito, queria, d...",Segurança Urbana,ultimo vou vou colocar escrito queria dizer su...
3223,"[pavimentacao, ruas, extremo, leste, itaquera,...",Zeladoria Urbana e melhorias de bairro,pavimentacao ruas extremo leste itaquera pavim...


In [82]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

# Baixar pacotes necessários do NLTK
nltk.download("rslp")

[nltk_data] Downloading package rslp to /home/h-pgy/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [83]:
# Criar Stemmer para português
stemmer = RSLPStemmer()

def stem_text(text):
    words = word_tokenize(text.lower())  # Tokeniza e converte para minúsculas
    return " ".join([stemmer.stem(word) for word in words if word.isalnum()])

In [84]:
#df['corpora'] = df['corpora'].apply(stem_text)

In [85]:
sample_series(df['corpora'])

'relacao construcoes unidades aqui regiao ubs banda coelho morais rua sepetiba dinheiro bid construcao projeto'

In [86]:
df['corpora'].duplicated().any()

np.True_

In [87]:
df['corpora'].duplicated().sum()

np.int64(71)

precisa dropar as duplicadas - está confundindo o modelo uma mesma frase ter mais de uma classificacao

In [88]:
df = df.drop_duplicates('corpora')

In [89]:
df['id'] = df.index

In [90]:
vectorizer = CountVectorizer()

In [91]:
bow_matrix= vectorizer.fit_transform(df['corpora'])

In [92]:
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [93]:
bow_df.head()

,1000km,100km,10ampliar,10km,119c10201210,11o,12e3,12milhoes,12o,1350km,...,zilda,zildo,zituo,zn,zogbi,zona,zonas,zoneamento,zr1,zum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
df.shape

(3991, 4)

In [95]:
bow_df.shape

(3991, 15385)

In [96]:
bow_df.head()

,1000km,100km,10ampliar,10km,119c10201210,11o,12e3,12milhoes,12o,1350km,...,zilda,zildo,zituo,zn,zogbi,zona,zonas,zoneamento,zr1,zum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
bow_df['y'] = df['categoria']

In [98]:
bow_df['corpora'] = df['corpora']

In [99]:
save_df_parquet(bow_df, 'bag_of_words_df.parquet')

'../data/bag_of_words_df.parquet'